In [45]:
import re

_hex_re = re.compile(r"^0[xX][0-9A-Fa-f]+$")

def ishex(s: str) -> bool:
    return bool(_hex_re.match(s))

In [46]:
class Int32:
    def __init__(self, value):
        """
        Initialize the Int32 instance.
        The value is stored as a 32-bit two's complement number.
        """
        if isinstance(value, Int32):
            self.value = value.value
        elif isinstance(value, int):
            self.value = value & 0xFFFFFFFF  # Ensure 32-bit representation
        else:
            raise TypeError("Unsupported type for Int32")

    def to_signed(self):
        """
        Return the signed integer representation.
        If the highest bit (bit 31) is set, the number is negative.
        """
        if self.value & 0x80000000:
            return self.value - 0x100000000
        return self.value

    def __add__(self, other):
        """
        Overload addition to work modulo 2^32.
        """
        if not isinstance(other, Int32):
            other = Int32(other)
        result = (self.value + other.value) & 0xFFFFFFFF
        return Int32(result)

    def __sub__(self, other):
        """
        Overload subtraction to work modulo 2^32.
        """
        if not isinstance(other, Int32):
            other = Int32(other)
        result = (self.value - other.value) & 0xFFFFFFFF
        return Int32(result)

    def __mod__(self, other):
        """
        Overload the modulus operator (%) to perform signed modulo.
        It uses the same division semantics as signed_div (truncation toward zero).
        Raises ZeroDivisionError if other is 0.
        """
        if not isinstance(other, Int32):
            other = Int32(other)
        # Utilize the signed_div method which computes quotient via truncation
        remainder, _ = self.signed_div(other)
        return remainder

    def __lt__(self, other):
        """
        Overload the less-than operator (<) based on signed value comparison.
        """
        if not isinstance(other, Int32):
            other = Int32(other)
        return self.to_signed() < other.to_signed()

    def __le__(self, other):
        """
        Overload the less-than-or-equal operator (<=) based on signed value comparison.
        """
        if not isinstance(other, Int32):
            other = Int32(other)
        return self.to_signed() <= other.to_signed()

    def __repr__(self):
        """
        String representation shows the signed value.
        """
        return f"Int32({self.to_signed()})"

    def signed_mult(self, other):
        """
        Perform signed multiplication.
        Multiply self and other (interpreted as signed 32-bit integers),
        then split the 64-bit product into hi (upper 32 bits) and lo (lower 32 bits).
        Returns a tuple: (hi, lo) as Int32 objects.
        """
        if not isinstance(other, Int32):
            other = Int32(other)
        a = self.to_signed()
        b = other.to_signed()
        product = a * b
        # Represent product as a 64-bit two's complement number
        product_64 = product & 0xFFFFFFFFFFFFFFFF
        hi = (product_64 >> 32) & 0xFFFFFFFF
        lo = product_64 & 0xFFFFFFFF
        return (Int32(hi), Int32(lo))

    def unsigned_mult(self, other):
        """
        Perform unsigned multiplication.
        Multiply self and other (using their 32-bit unsigned values),
        then split the 64-bit product into hi and lo.
        Returns a tuple: (hi, lo) as Int32 objects.
        """
        if not isinstance(other, Int32):
            other = Int32(other)
        a = self.value
        b = other.value
        product = a * b
        hi = (product >> 32) & 0xFFFFFFFF
        lo = product & 0xFFFFFFFF
        return (Int32(hi), Int32(lo))

    def signed_div(self, divisor):
        """
        Perform signed division.
        Divide self by divisor (interpreted as signed 32-bit numbers).
        The quotient is truncated toward zero.
        Returns a tuple (hi, lo) where hi is the remainder and lo is the quotient.
        Raises ZeroDivisionError if divisor is 0.
        """
        if not isinstance(divisor, Int32):
            divisor = Int32(divisor)
        a = self.to_signed()
        b = divisor.to_signed()
        if b == 0:
            raise ZeroDivisionError("Division by zero")
        # Truncate quotient toward zero (using int() conversion)
        quotient = int(a / b)
        remainder = a - quotient * b
        return (Int32(remainder), Int32(quotient))

    def unsigned_div(self, divisor):
        """
        Perform unsigned division.
        Divide self by divisor (using their 32-bit unsigned values).
        Returns a tuple (hi, lo) where hi is the remainder and lo is the quotient.
        Raises ZeroDivisionError if divisor is 0.
        """
        if not isinstance(divisor, Int32):
            divisor = Int32(divisor)
        a = self.value
        b = divisor.value
        if b == 0:
            raise ZeroDivisionError("Division by zero")
        quotient = a // b
        remainder = a % b
        return (Int32(remainder), Int32(quotient))
    
    def unsigned_le(self, other):
        """
        Perform unsigned less-than-or-equal comparison.
        """
        if not isinstance(other, Int32):
            other = Int32(other)
        return self.value <= other.value


In [47]:
a = Int32(10)
b = Int32(3)

# Addition and subtraction
print("Addition:", a + b)           # Int32(13)
print("Subtraction:", a - b)        # Int32(7)

# Signed multiplication
a_neg = Int32(-2)
hi, lo = a_neg.signed_mult(Int32(3))
print("Signed Multiplication (-2 * 3):")
print("  HI (upper 32 bits):", hi)   # hi should represent the sign extension (typically -1)
print("  LO (lower 32 bits):", lo)   # lo should represent -6 in 32-bit two's complement

# Unsigned multiplication
hi_u, lo_u = a_neg.unsigned_mult(Int32(3))
print("Unsigned Multiplication (0xFFFFFFFE * 3):")
print("  HI (upper 32 bits):", hi_u)
print("  LO (lower 32 bits):", lo_u)

# Signed division (quotient in LO, remainder in HI)
rem, quot = a.signed_div(b)
print("Signed Division (10 / 3):")
print("  Remainder (HI):", rem)
print("  Quotient (LO):", quot)

# Unsigned division
rem_u, quot_u = a.unsigned_div(b)
print("Unsigned Division (10 / 3):")
print("  Remainder (HI):", rem_u)
print("  Quotient (LO):", quot_u)


Addition: Int32(13)
Subtraction: Int32(7)
Signed Multiplication (-2 * 3):
  HI (upper 32 bits): Int32(-1)
  LO (lower 32 bits): Int32(-6)
Unsigned Multiplication (0xFFFFFFFE * 3):
  HI (upper 32 bits): Int32(2)
  LO (lower 32 bits): Int32(-6)
Signed Division (10 / 3):
  Remainder (HI): Int32(1)
  Quotient (LO): Int32(3)
Unsigned Division (10 / 3):
  Remainder (HI): Int32(1)
  Quotient (LO): Int32(3)


In [48]:
class MIPS:

    class instruction:
        def __init__(self, fn, val1, val2, val3, labels = set()):
            self.fn = fn
            self.val1 = val1
            self.val2 = val2
            self.val3 = val3
            self.labels = labels

        def __str__(self):
            return f'{self.fn} {self.val1} {self.val2} {self.val3}'
        
        
    def __init__(self):
        self.registers = [0] * 32
        self.pc = 0
        self.instructions = []
        self.labels = {}
        self.memory = {}
        self.hi = 0
        self.lo = 0


    def __add_label(self, labels, instr, lineNumber):
        # while there is a ':'
        while ':' in instr:
            # print(f"adding label: {instr[:instr.index(':')]} and string to {instr[instr.index(':')+1:]}")
            # get the label
            label = instr[:instr.index(':')]
            # remove label from the instring
            instr = instr[instr.index(':')+1:]
            # add the label to the labels dictionary and labels set
            self.labels[label] = lineNumber
            labels.add(label)

        return instr

    def __process_instruction(self, instr):
        initialInstr = instr

        # replace all ',', '(' and ')' with ' '
        instr = instr.replace(',', ' ')
        instr = instr.replace('(', ' ')
        instr = instr.replace(')', ' ')

        cleanedInstr = instr

        # split the instring into a list of instrings
        instr = instr.split()
        instr = [x.replace('$', '') for x in instr]

        # if the instruction parsed is invalid length
        if len(instr) == 0 or len(instr) > 4:
            raise Exception(f'Invalid instruction: {initialInstr} -> {cleanedInstr}')
        
        return instr

    def add_instruction(self, instr):

        # if instruction is empty, or only white space or start with ';'
        if len(instr) == 0 or instr.isspace() or instr[0] == ';':
            return

        labels = set()

        # print(f'processing instruction: {instr}')

        try:
            instr = self.__add_label(labels, instr, len(self.instructions))
            # print(f'after adding label: {instr}')
            instr = self.__process_instruction(instr)
        except Exception as e:
            print(f'LINE {len(self.instructions) + 1}: {e}')

        # self.instructions.append(self.instruction(instr[0], instr[1], instr[2], instr[3], labels))
        # use value only when exist

        # print(f'adding instruction: {instr}')

        try:
            fn = instr[0]
            val1 = instr[1] if len(instr) > 1 else 0
            val2 = instr[2] if len(instr) > 2 else 0
            val3 = instr[3] if len(instr) > 3 else 0
            self.instructions.append(self.instruction(fn, val1, val2, val3, labels))
        except Exception as e:
            print(f'LINE {len(self.instructions) + 1}: {e}')

    def __str__(self):
        str = ""

        for i in range(len(self.instructions)):
            for labels in self.instructions[i].labels:
                str += f'{labels}:\n'

            str += f'{i}| {self.instructions[i]}\n'

        return str

    def __runInstr(self, instr):
        if instr.fn == 'add':
            d = Int32(instr.val1)
            s = Int32(instr.val2)
            t = Int32(instr.val3)
            self.registers[d] = self.registers[s] + self.registers[t]


        elif instr.fn == 'sub':
            d = Int32(instr.val1)
            s = Int32(instr.val2)
            t = Int32(instr.val3)
            self.registers[d] = self.registers[s] - self.registers[t]

        elif instr.fn == 'multu':
            s = Int32(instr.val1)
            t = Int32(instr.val2)
            self.hi, self.lo = s.unsigned_mult(t)

        elif instr.fn == 'mult':
            s = Int32(instr.val1)
            t = Int32(instr.val2)
            self.hi, self.lo = s.signed_mult(t)

        elif instr.fn == 'divu':
            s = Int32(instr.val1)
            t = Int32(instr.val2)
            self.hi, self.lo = s.unsigned_div(t)

        elif instr.fn == 'div':
            s = Int32(instr.val1)
            t = Int32(instr.val2)
            self.hi, self.lo = s.signed_div(t)

        elif instr.fn == 'mfhi':
            d = Int32(instr.val1)
            self.registers[d] = self.hi

        elif instr.fn == 'mflo':
            d = Int32(instr.val1)
            self.registers[d] = self.lo

        elif instr.fn == 'lis':
            d = Int32(instr.val1)
            if self.pc > 0:
                self.registers[d] = self.instructions[self.pc / 4 - 1].val1
            else:
                raise Exception('lis cannot be the first instruction')
            
        elif instr.fn == 'lw':
            t = Int32(instr.val1)
            i = Int32(instr.val2)
            s = Int32(instr.val3)
            loc = self.registers[s] + i
            
            if loc % 4 != 0:
                raise Exception(f'Address {loc} not divisible by 4')
            
            if loc in self.memory:
                self.registers[t] = self.memory[loc]
            else:
                raise Exception(f'Address {loc} not found in memory')
            
        elif instr.fn == 'sw':
            t = Int32(instr.val1)
            i = Int32(instr.val2)
            s = Int32(instr.val3)

            loc = self.registers[s] + i
            
            if loc % 4 != 0:
                raise Exception(f'Address {loc} not divisible by 4')

            self.memory[loc] = self.registers[t]

        elif instr.fn == 'slt':
            d = Int32(instr.val1)
            s = Int32(instr.val2)
            t = Int32(instr.val3)
            self.registers[d] = 1 if self.registers[s] < self.registers[t] else 0

        elif instr.fn == 'sltu':
            d = Int32(instr.val1)
            s = Int32(instr.val2)
            t = Int32(instr.val3)
            self.registers[d] = 1 if self.registers[s].unsigned_le(self.registers[t]) else 0

        elif instr.fn == 'beq':
            s = Int32(instr.val1)
            t = Int32(instr.val2)
            i = Int32(instr.val3)
            if self.registers[s] == self.registers[t]:
                self.pc += i.to_signed() * 4

        elif instr.fn == 'bne':
            s = Int32(instr.val1)
            t = Int32(instr.val2)
            i = Int32(instr.val3)
            if self.registers[s] != self.registers[t]:
                self.pc += i.to_signed() * 4

        elif instr.fn == 'jr':
            s = Int32(instr.val1)
            self.pc = self.registers[s]

        elif instr.fn == 'jalr':
            s = Int32(instr.val1)
            d = Int32(32)
            self.registers[d] = self.pc
            self.pc = self.registers[s]

        elif instr.fn == 'addi':
            t = Int32(instr.val1)
            s = Int32(instr.val2)
            i = Int32(instr.val3)
            self.registers[t] = self.registers[s] + i

        elif instr.fn == 'j':
            i = Int32(instr.val1)
            self.pc = i.value * 4

        elif instr.fn == 'jal':
            i = Int32(instr.val1)
            d = Int32(31)
            self.registers[d] = self.pc
            self.pc = i.value * 4

        elif instr.fn == '.word':
            pass

        else:
            raise Exception(f'Instruction {instr.fn} not found')

    def execute(self, pc = 0):
        self.pc = pc
        while 0 <= self.pc < len(self.instructions) * 4:
            instr = self.instructions[self.pc]
            self.__runInstr(instr)

        print(f'Execution complete. PC: {self.pc}')
        print(f'Final Registers: {self.registers}')
        print(f'Final Memory: {self.memory}')
        
    def compile(self):
        """
        Compiles each instruction to MIPS assembly code.
        Returns a string representation of the compiled code.
        """

        compiled_code = []
        for instr in self.instructions:
            if instr.fn == 'add':
                d = f'{int(instr.val1):05b}'
                s = f'{int(instr.val2):05b}'
                t = f'{int(instr.val3):05b}'
                compiled_code.append(f'000000{s}{t}{d}00000100000')
            elif instr.fn == 'sub':
                d = f'{int(instr.val1):05b}'
                s = f'{int(instr.val2):05b}'
                t = f'{int(instr.val3):05b}'
                compiled_code.append(f'000000{s}{t}{d}00000100010')
            elif instr.fn == 'mult':
                s = f'{int(instr.val1):05b}'
                t = f'{int(instr.val2):05b}'
                compiled_code.append(f'000000{s}{t}0000000000011000')
            elif instr.fn == 'multu':
                s = f'{int(instr.val1):05b}'
                t = f'{int(instr.val2):05b}'
                compiled_code.append(f'000000{s}{t}0000000000011001')
            elif instr.fn == 'div':
                s = f'{int(instr.val1):05b}'
                t = f'{int(instr.val2):05b}'
                compiled_code.append(f'000000{s}{t}0000000000011010')
            elif instr.fn == 'divu':
                s = f'{int(instr.val1):05b}'
                t = f'{int(instr.val2):05b}'
                compiled_code.append(f'000000{s}{t}0000000000011011')
            elif instr.fn == 'mfhi':
                d = f'{int(instr.val1):05b}'
                compiled_code.append(f'0000000000000000{d}00000010000')
            elif instr.fn == 'mflo':
                d = f'{int(instr.val1):05b}'
                compiled_code.append(f'0000000000000000{d}00000010010')
            elif instr.fn == 'lis':
                d = f'{int(instr.val1):05b}'
                compiled_code.append(f'0000000000000000{d}00000010100')
            elif instr.fn == 'lw':
                t = f'{int(instr.val1):05b}'
                i = f'{int(instr.val2):016b}'
                s = f'{int(instr.val3):05b}'
                compiled_code.append(f'100011{s}{t}{i}')
            elif instr.fn == 'sw':
                t = f'{int(instr.val1):05b}'
                i = f'{int(instr.val2):016b}'
                s = f'{int(instr.val3):05b}'
                compiled_code.append(f'101011{s}{t}{i}')
            elif instr.fn == 'slt':
                d = f'{int(instr.val1):05b}'
                s = f'{int(instr.val2):05b}'
                t = f'{int(instr.val3):05b}'
                compiled_code.append(f'000000{s}{t}{d}00000101010')
            elif instr.fn == 'sltu':
                d = f'{int(instr.val1):05b}'
                s = f'{int(instr.val2):05b}'
                t = f'{int(instr.val3):05b}'
                compiled_code.append(f'000000{s}{t}{d}00000101011')
            elif instr.fn == 'beq':
                s = f'{int(instr.val1):05b}'
                t = f'{int(instr.val2):05b}'
                i = None
                if isinstance(instr.val3, int) or instr.val3.isdigit():
                    i = f'{int(instr.val3):016b}'
                elif instr.val3 in self.labels:
                    i = f'{int(self.labels[instr.val3]):016b}'
                elif ishex(instr.val3):
                    i = f'{int(instr.val3, 16):016b}'
                else:
                    raise Exception(f'Label {instr.val3} has invalid format')
                compiled_code.append(f'000100{s}{t}{i}')
            elif instr.fn == 'bne':
                s = f'{int(instr.val1):05b}'
                t = f'{int(instr.val2):05b}'
                i = None
                if isinstance(instr.val3, int) or instr.val3.isdigit():
                    i = f'{int(instr.val3):016b}'
                elif instr.val3 in self.labels:
                    i = f'{int(self.labels[instr.val3]):016b}'
                elif ishex(instr.val3):
                    i = f'{int(instr.val3, 16):016b}'
                else:
                    raise Exception(f'Label {instr.val3} has invalid format')
                compiled_code.append(f'000101{s}{t}{i}')
            elif instr.fn == 'jr':
                s = f'{int(instr.val1):05b}'
                compiled_code.append(f'000000{s}000000000000000001000')
            elif instr.fn == 'jalr':
                s = f'{int(instr.val1):05b}'
                compiled_code.append(f'000000{s}000000000000000001001')
            elif instr.fn == 'addi':
                d = f'{int(instr.val1):05b}'
                s = f'{int(instr.val2):05b}'
                i = f'{int(instr.val3):016b}'
                compiled_code.append(f'001000{s}{d}{i}')
            elif instr.fn == '.word':
                i = None
                if isinstance(instr.val1, int) or instr.val1.isdigit():
                    i = f'{int(instr.val1):032b}'
                elif ishex(instr.val1):
                    i = f'{int(instr.val1, 16):032b}'
                elif instr.val1 in self.labels:
                    i = f'{self.labels[instr.val1]:032b}'
                else:
                    raise Exception(f'Invalid .word value: {instr.val1}')
                compiled_code.append(f'{i}')
            else:
                raise Exception(f'Instruction {instr.fn} not found')
        return '\n'.join(compiled_code)

    def print_labels(self):
        """
        Print the labels and their corresponding line numbers.
        """
        for label, line_number in self.labels.items():
            print(f'{label}: {line_number}')
            




    
        

                

In [49]:
# filename = input("Enter the filename: ")
filename = "load.asm"

lines = []
label = ""
with open(filename, 'r') as file:
    for line in file:

        line = line.strip()

        # if line ends with :

        if line.endswith(":"):
            label += line
            continue
        else:
            lines.append(label + line)
            label = ""

# print(lines)

mips = MIPS()

for line in lines:
    mips.add_instruction(line)

print(mips)

mips.print_labels()






prologue:
0| lis 4 0 0
1| .word 4 0 0
2| add 11 0 0
3| lis 22 0 0
4| .word readWord 0 0
5| lis 23 0 0
6| .word printHex 0 0
7| sw 31 -4 30
8| sub 30 30 4
9| jalr 22 0 0
10| add 12 3 0
11| jalr 22 0 0
12| sub 11 11 4
13| jalr 22 0 0
14| sub 11 11 4
15| jalr 22 0 0
16| sub 11 11 4
17| add 11 11 3
18| add 5 11 0
19| add 6 12 0
FirstPrint:
20| jalr 22 0 0
21| sub 5 5 4
22| add 1 3 0
23| sw 1 0 6
24| add 6 6 4
25| jalr 23 0 0
26| beq 5 0 AfterFirstPrint
27| beq 0 0 FirstPrint
AfterFirstPrint:
28| add 5 11 0
29| add 6 12 0
Secondprinting:
30| lw 1 0 6
31| add 6 6 4
32| sub 5 5 4
33| jalr 23 0 0
34| beq 5 0 epilogue
35| beq 0 0 Secondprinting
epilogue:
36| add 30 30 4
37| lw 31 -4 30
38| jr 31 0 0
readWord:
39| sw 1 -4 30
40| sw 2 -8 30
41| sw 4 -12 30
42| sw 5 -16 30
43| sw 6 -20 30
44| sw 7 -24 30
45| sw 8 -28 30
46| lis 8 0 0
47| .word 28 0 0
48| sub 30 30 8
49| lis 4 0 0
50| .word 0x01000000 0 0
51| lis 3 0 0
52| .word 0x00010000 0 0
53| lis 2 0 0
54| .word 0x00000100 0 0
55| lis 1 0 0
56

In [50]:

print(mips.compile())

00000000000000000010000000010100
00000000000000000000000000000100
00000000000000000101100000100000
00000000000000001011000000010100
00000000000000000000000000100111
00000000000000001011100000010100
00000000000000000000000001010001
1010111111011111-000000000000100
00000011110001001111000000100010
00000010110000000000000000001001
00000000011000000110000000100000
00000010110000000000000000001001
00000001011001000101100000100010
00000010110000000000000000001001
00000001011001000101100000100010
00000010110000000000000000001001
00000001011001000101100000100010
00000001011000110101100000100000
00000001011000000010100000100000
00000001100000000011000000100000
00000010110000000000000000001001
00000000101001000010100000100010
00000000011000000000100000100000
10101100110000010000000000000000
00000000110001000011000000100000
00000010111000000000000000001001
00010000101000000000000000011100
00010000000000000000000000010100
00000001011000000010100000100000
00000001100000000011000000100000
1000110011